In [1]:
import pickle

with open("/workspace/exploration-hacking/post_benign_eval_3_results.pkl", "rb") as f:
    results = pickle.load(f)

In [2]:
import numpy as np

np.mean(results.metrics["accuracy"])

np.float64(0.8117647058823529)

In [8]:
results.metrics.keys()

dict_keys(['accuracy', 'format_penalty_func', 'completion_under_length_penalty', 'completion_over_length_penalty', 'response_under_length_penalty', 'response_over_length_penalty', 'total_tool_calls', 'search_web_calls'])

In [12]:
import mlflow

In [13]:
def _log_to_mlflow(results, name):
    try:
        mlflow.create_experiment(name)
    except Exception as e:
        print(f"Error creating MLFlow experiment: {e}")

    mlflow.set_experiment(name)

    for i in range(len(results.prompt)):
        print(f"Logging trace {i}")
        inputs = {"prompt": results.prompt[i]}
        tags = {k: str(v[i]) for k, v in results.metrics.items()}
        tags |= {"reward": str(results.reward[i])}
        span = mlflow.start_span_no_context("generation", inputs=inputs, tags=tags)
        try:
            span.set_outputs(
                {"completion": results.completion[i], "answer": results.answer[i]}
            )
        finally:
            span.end()

In [9]:
mlflow.config.enable_async_logging()

In [14]:
import random

random.seed(42)

In [17]:
from contextlib import contextmanager

@contextmanager
def true_random_context():
    """Temporarily use true randomness, then restore previous state."""
    saved_state = random.getstate()
    try:
        random.seed()  # Reseed with current time
        yield
    finally:
        random.setstate(saved_state)

In [19]:
with true_random_context():
    _log_to_mlflow(results, "post_benign_eval_3_test6")

Logging trace 0
Logging trace 1
Logging trace 2
Logging trace 3


KeyboardInterrupt: 

[Trace(trace_id=tr-781727dee423127236b71ae801955f58), Trace(trace_id=tr-3784154250904aee2db994f08beededb), Trace(trace_id=tr-bec8400de3cb88eb9a9e4cb0074aea02), Trace(trace_id=tr-d9d09a42da9427c5495444b5d0c974de)]